# 🚀 Inférence LLaMA + LoRA - Notebook Colab Prêt à l'Emploi

Ce notebook teste l'inférence avec votre modèle LLaMA fine-tuné avec LoRA.

## 📋 Instructions rapides

1. **Activer le GPU** : Runtime → Change runtime type → GPU (T4 ou A100)
2. **Remplacer les chemins** dans les cellules ci-dessous (marqués avec ⚠️)
3. **Exécuter les cellules** dans l'ordre (Shift+Enter)

## 📝 Checklist

- [ ] GPU activé
- [ ] Token Hugging Face prêt
- [ ] Modèle LoRA disponible (Google Drive, Hugging Face, ou upload)
- [ ] PDF de test + template JSON prêts


## ÉTAPE 1 : Vérifier le GPU


In [ ]:
import torch
print(f"GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("❌ ATTENTION: Aucun GPU détecté!")
    print("   Allez dans Runtime → Change runtime type → GPU (T4 ou A100)")


## ÉTAPE 2 : Installer les dépendances depuis requirements.txt


In [ ]:
# Option A: Télécharger requirements.txt depuis votre repo GitHub
# Décommentez et modifiez l'URL si vous avez le fichier sur GitHub
# !wget -q https://raw.githubusercontent.com/VOTRE_USERNAME/VOTRE_REPO/main/amalytics-ml/code/requirements.txt -O /content/requirements.txt

# Option B: Créer requirements.txt directement avec toutes les dépendances
requirements_content = """torch
transformers
accelerate
bitsandbytes
peft
datasets
trl
pdfplumber
pyffx
scikit-learn
natsort
fpdf2
lxml
nltk
huggingface_hub"""

# Vérifier si requirements.txt existe déjà, sinon le créer
import os
req_file = "/content/requirements.txt"
if not os.path.exists(req_file):
    with open(req_file, "w") as f:
        f.write(requirements_content)
    print("✅ requirements.txt créé localement")
else:
    print("✅ requirements.txt trouvé (probablement téléchargé depuis GitHub)")

print("\n📦 Installation des dépendances depuis requirements.txt...")
print("   (Cela peut prendre 5-10 minutes, soyez patient...)")

# Installer depuis requirements.txt
!pip install -q -r /content/requirements.txt

print("\n✅ Toutes les dépendances sont installées!")
print("\n📋 Packages installés:")
!pip list | grep -E "(torch|transformers|peft|bitsandbytes|pdfplumber)"


## ÉTAPE 3 : Authentification Hugging Face


In [ ]:
from huggingface_hub import login

# ⚠️ REMPLACER PAR VOTRE TOKEN Hugging Face
# Obtenez votre token sur: https://huggingface.co/settings/tokens
HF_TOKEN = "VOTRE_TOKEN_HF"  # Token commençant par hf_...

login(HF_TOKEN)
print("✅ Authentification Hugging Face réussie")


## ÉTAPE 4 : Télécharger/Uploadez votre code


In [ ]:
import sys
from pathlib import Path
import os

# ============================================
# OPTION A : Cloner depuis GitHub (RECOMMANDÉ)
# ============================================
# ⚠️ MODIFIER : Remplacez par l'URL de votre repository GitHub
GITHUB_REPO_URL = "https://github.com/VOTRE_USERNAME/amalytics-ml.git"  # ⚠️ MODIFIER

# Si le repo est private, utilisez un token:
# GITHUB_REPO_URL = "https://VOTRE_TOKEN@github.com/VOTRE_USERNAME/amalytics-ml.git"

# Cloner depuis GitHub
if not os.path.exists("/content/amalytics-ml"):
    print(f"📥 Clonage depuis GitHub: {GITHUB_REPO_URL}")
    !git clone {GITHUB_REPO_URL} /content/amalytics-ml
    print("✅ Code cloné depuis GitHub!")
else:
    print("✅ Le repository existe déjà (peut-être déjà cloné)")

# ============================================
# OPTION B : Upload manuel (si vous n'utilisez pas GitHub)
# ============================================
# Décommentez cette section si vous préférez uploader manuellement
# !mkdir -p /content/amalytics-ml/code
# print("✅ Dossier créé")
# print("\n💡 Uploadez maintenant votre code via l'interface Files (📁) de Colab")
# print("   dans le dossier /content/amalytics-ml/code/")

# Ajouter le code au path Python
code_dir = Path("/content/amalytics-ml/code")
if not code_dir.exists():
    # Si le repo GitHub a une structure différente, ajustez ici
    code_dir = Path("/content/amalytics-ml")

src_dir = code_dir / "src"
if src_dir.exists():
    sys.path.insert(0, str(src_dir))
    print(f"✅ Code ajouté au path: {src_dir}")
else:
    print(f"⚠️ Dossier src non trouvé dans {code_dir}")
    print("   Vérifiez la structure de votre repository GitHub")

# Vérifier la structure
print(f"\n📁 Structure du repository:")
for item in sorted(code_dir.rglob("*"))[:15]:  # Afficher les 15 premiers
    if item.is_file() and item.suffix in ['.py', '.json', '.txt', '.md']:
        print(f"  📄 {item.relative_to(code_dir)}")


## ÉTAPE 5 : Vérifier les imports


In [ ]:
# Vérifier que le code peut être importé
try:
    from amalytics_ml.config import InferenceConfig
    from amalytics_ml.models.inference import run_inference
    print("✅ Tous les imports fonctionnent!")
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    print("\n💡 Assurez-vous que:")
    print("   - Le code est bien uploadé dans /content/amalytics-ml/code/")
    print("   - La structure est: /content/amalytics-ml/code/src/amalytics_ml/...")
    import traceback
    traceback.print_exc()


## ÉTAPE 6 : Télécharger votre modèle LoRA


### Option A : Depuis Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ⚠️ MODIFIER LE CHEMIN selon votre structure sur Google Drive
LORA_DRIVE_PATH = "/content/drive/MyDrive/path/to/lora-output"  # ⚠️ MODIFIER

!cp -r {LORA_DRIVE_PATH} /content/lora-output

print("✅ LoRA copié depuis Google Drive")


### Option B : Depuis Hugging Face Hub


In [ ]:
from huggingface_hub import snapshot_download

# ⚠️ REMPLACER par votre repo Hugging Face
LORA_REPO_ID = "VOTRE_USERNAME/VOTRE_LORA_REPO"  # ⚠️ MODIFIER

snapshot_download(
    repo_id=LORA_REPO_ID,
    local_dir="/content/lora-output",
    token=HF_TOKEN
)

print("✅ LoRA téléchargé depuis Hugging Face")


### Option C : Upload direct


In [ ]:
!mkdir -p /content/lora-output
print("✅ Dossier créé")
print("\n💡 Uploadez maintenant vos fichiers LoRA via l'interface Files (📁) de Colab")
print("   dans le dossier /content/lora-output/")


### Vérifier que les fichiers LoRA sont présents


In [ ]:
import os
lora_path = "/content/lora-output"
if os.path.exists(lora_path):
    files = os.listdir(lora_path)
    print(f"✅ {len(files)} fichiers trouvés dans {lora_path}")
    print("\nFichiers LoRA:")
    for f in sorted(files)[:10]:
        print(f"  - {f}")
    if len(files) > 10:
        print(f"  ... et {len(files) - 10} autres fichiers")
    
    # Vérifier les fichiers importants
    important_files = ["adapter_config.json", "adapter_model.bin", "adapter_model.safetensors"]
    found_important = [f for f in important_files if any(f in file for file in files)]
    if found_important:
        print(f"\n✅ Fichiers LoRA importants trouvés: {', '.join(found_important)}")
else:
    print("❌ Dossier LoRA non trouvé!")
    print(f"   Vérifiez que {lora_path} existe")


## ÉTAPE 7 : Préparer les fichiers de test


In [ ]:
!mkdir -p /content/test_data/templates
print("✅ Dossiers créés")
print("\n📁 Uploadez maintenant via l'interface Files (📁) de Colab:")
print("   - Un PDF de test → /content/test_data/")
print("   - Un template JSON vide → /content/test_data/templates/")

# Vérifier les fichiers uploadés
import os
test_dir = "/content/test_data"
if os.path.exists(test_dir):
    pdfs = [f for f in os.listdir(test_dir) if f.endswith('.pdf')]
    templates = []
    if os.path.exists(f"{test_dir}/templates"):
        templates = [f for f in os.listdir(f"{test_dir}/templates") if f.endswith('.json')]
    
    print(f"\n📄 Fichiers trouvés:")
    print(f"   PDFs: {len(pdfs)} - {pdfs[:3]}")
    print(f"   Templates: {len(templates)} - {templates[:3]}")


## ÉTAPE 8 : Configuration


In [ ]:
import json
from pathlib import Path
from amalytics_ml.config import InferenceConfig

# ⚠️ MODIFIER LES CHEMINS selon votre structure
config = {
    "model_path": "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "lora_path": "/content/lora-output",
    "template_path": "/content/test_data/templates/sample_1_template_empty.json",  # ⚠️ MODIFIER
    "max_new_tokens": 3000,
    "do_sample": False,
    "return_scores": True,
    "load_in_4bit": True,
    "device_map": "auto",
    "use_batch_inference": False,
    "apply_anonymization": False,
}

cfg = InferenceConfig(**config)

# Charger le template
if cfg.template_path:
    template_path = Path(cfg.template_path)
    if template_path.exists():
        with template_path.open("r", encoding="utf-8") as f:
            cfg.template = json.load(f)
        print(f"✅ Template chargé: {cfg.template_path}")
        print(f"   Clés principales: {list(cfg.template.keys())[:5]}")
    else:
        print(f"❌ Template non trouvé: {cfg.template_path}")
        print("   Vérifiez le chemin dans la configuration ci-dessus")

print(f"\n✅ Configuration prête")
print(f"   Model: {cfg.model_path}")
print(f"   LoRA: {cfg.lora_path}")
print(f"   4-bit quantization: {cfg.load_in_4bit}")
print(f"   Device: {cfg.device_map}")


## ÉTAPE 9 : 🚀 EXÉCUTER L'INFÉRENCE

⚠️ **ATTENTION** : Cette étape va charger le modèle en mémoire GPU (5-15 minutes la première fois)


In [ ]:
from amalytics_ml.models.inference import run_inference
from pathlib import Path
import json
import time

# ⚠️ MODIFIER LE CHEMIN vers votre PDF de test
pdf_path = "/content/test_data/sample_1.pdf"  # ⚠️ MODIFIER

print("🚀 Démarrage de l'inférence...")
print("⏳ Chargement du modèle (cela peut prendre 5-15 minutes)...")
print("💡 Vous pouvez surveiller l'utilisation GPU dans Runtime → Manage sessions")
print("-" * 60)

start_time = time.time()

try:
    result = run_inference(
        model_path=cfg.model_path,
        lora_path=cfg.lora_path if cfg.lora_path else "",
        input_text=Path(pdf_path),
        config=cfg,
    )
    
    elapsed_time = time.time() - start_time
    
    print("-" * 60)
    print(f"✅ Inférence terminée avec succès! (temps: {elapsed_time/60:.2f} minutes)")
    
except Exception as e:
    elapsed_time = time.time() - start_time
    print("-" * 60)
    print(f"❌ Erreur pendant l'inférence: {e}")
    print(f"   Temps écoulé: {elapsed_time/60:.2f} minutes")
    import traceback
    traceback.print_exc()


## ÉTAPE 10 : Afficher les résultats


In [ ]:
# Afficher le résultat JSON
print("=" * 60)
print("RÉSULTAT JSON")
print("=" * 60)
print(json.dumps(result.parsed_json, indent=2, ensure_ascii=False))


In [ ]:
# Afficher les scores de confiance
if result.confidence_scores:
    print("\n" + "=" * 60)
    print("SCORES DE CONFIANCE")
    print("=" * 60)
    print(json.dumps(result.confidence_scores, indent=2, ensure_ascii=False))
    
    # Statistiques
    scores = list(result.confidence_scores.values())
    print(f"\n📊 Statistiques:")
    print(f"   Nombre de champs: {len(scores)}")
    print(f"   Score moyen: {sum(scores) / len(scores):.4f}")
    print(f"   Score min: {min(scores):.4f}")
    print(f"   Score max: {max(scores):.4f}")
else:
    print("\n⚠️ Pas de scores de confiance disponibles (return_scores=False)")


## ÉTAPE 11 : Sauvegarder les résultats


In [ ]:
output_dir = Path("/content/outputs")
output_dir.mkdir(exist_ok=True)

# Sauvegarder le résultat JSON
result_file = output_dir / "result.json"
with result_file.open("w", encoding="utf-8") as f:
    json.dump(result.parsed_json, f, indent=2, ensure_ascii=False)
print(f"✅ Résultat sauvegardé: {result_file}")

# Sauvegarder les scores de confiance si disponibles
if result.confidence_scores:
    confidence_file = output_dir / "confidence.json"
    with confidence_file.open("w", encoding="utf-8") as f:
        json.dump(result.confidence_scores, f, indent=2, ensure_ascii=False)
    print(f"✅ Scores sauvegardés: {confidence_file}")

print(f"\n💡 Pour télécharger:")
print(f"   Clic droit sur les fichiers dans l'interface Files (📁) → Download")
print(f"   Ou utilisez: files.download('{result_file}')")


## 📊 Visualisation des scores (optionnel)


In [ ]:
if result.confidence_scores:
    import matplotlib.pyplot as plt
    
    scores = list(result.confidence_scores.values())
    
    plt.figure(figsize=(10, 6))
    plt.hist(scores, bins=20, edgecolor='black', alpha=0.7)
    plt.xlabel('Score de confiance', fontsize=12)
    plt.ylabel('Nombre de champs', fontsize=12)
    plt.title('Distribution des scores de confiance', fontsize=14)
    plt.grid(True, alpha=0.3)
    avg_score = sum(scores) / len(scores)
    plt.axvline(avg_score, color='r', linestyle='--', label=f'Moyenne: {avg_score:.3f}')
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Pas de scores de confiance pour visualiser")
